In [81]:
library(dplyr)
library(tidyr)
library(tibble)
library(qvalue)
library(data.table)

source("/mnt/lareaulab/reliscu/code/fisher_test.R")

setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM")

In [82]:
get_module_enrichments <- function(network_dir, ctype_genes, mod_def="PosFDR") {
    # Traverse networks to get cell type enrichments for each module
    networks <- list.dirs(file.path(getwd(), network_dir), full.names=TRUE, recursive=FALSE)
    networks <- networks[lengths(lapply(networks, list.files)) > 0]

    enrichments_list <- lapply(seq_along(networks), function(i) {
        kME_path <- list.files(networks[i])[grep("kME", list.files(networks[i]))]
        kME <- fread(file.path(networks[i], kME_path), data.table=FALSE)
        mod_col <- grep("PosFDR", colnames(kME))
        mod_genes <- tapply(kME$Gene, kME[,mod_col], list)
        
        if (length(mod_genes) > 0) {
            all_genes <- kME$Gene

            # For each module: calculate enrichment for DE genes from each cell type
            mod_enrichments_list <- lapply(mod_genes, function(mod) {
                lapply(unlist(lapply(ctype_genes, function(set) {
                    fisher_test(set, mod, all=all_genes)
                })), c)
            })
            
            # Save the network the module came from
            network_id <- sapply(strsplit(networks[i], "/"), function(x) x[length(x)])
            mod_enrichments_df <- reshape2::melt(mod_enrichments_list)
            colnames(mod_enrichments_df) <- c("Pval", "Cell_type", "Module")
            
            # Save path to module eigengenes table for downstream analyses
            ME_path <- list.files(networks[i])[grep("eigengene", list.files(networks[i]))]
            
            data.frame(
                Network=network_id,
                kME_path=file.path(networks[i], kME_path),
                ME_path=file.path(networks[i], ME_path),
                mod_enrichments_df
            )
        }
    })
    enrichments_df <- do.call(rbind, enrichments_list)
    enrichments_df$Qval <- qvalue(enrichments_df$Pval)$qvalue

    enrichments_df
}

Here I perform enrichment analysis to find modules enriched for cell type markers. 

These modules will later be used to correlate with exon PSI to define cell type-specific exons.

In [83]:
network_dir <- "tasic_2018_ALM_STAR_20pcntCells_30pcntVar_200samples_log2_pseudobulk_Modules"
network_dataset <- "20pcntCells_30pcntVar_200samples_log2_pseudobulk"

In [84]:
mod_def <- "PosBC"
unique <- TRUE

### Prep DE genes

#### 1 vs. pooled tests

In [85]:
pooled_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream.RDS")

In [86]:
pval_threshold <- .05/length(pooled_res_list)
lfc_threshold <- 6

pooled_ctype_genes <- lapply(pooled_res_list, function(df) {
    mask <- (df['adj.P.Val'] < pval_threshold) & (abs(df['logFC']) > lfc_threshold)
    df[mask, 1]
})
names(pooled_ctype_genes) <- names(pooled_res_list) 

In [87]:
pooled_ctype_genes

$Astro
  [1] "Gpr37l1"       "Slc7a10"       "Aqp4"          "Lcat"         
  [5] "Gja1"          "Slc1a3"        "Slco1c1"       "Gjb6"         
  [9] "Acsbg1"        "Ntsr2"         "Cyp4f15"       "Mlc1"         
 [13] "S1pr1"         "Gm6145"        "Fgfr3"         "Slc39a12"     
 [17] "Cldn10"        "F3"            "Nwd1"          "Slc25a18"     
 [21] "Btbd17"        "App"           "Atp13a4"       "Cbs"          
 [25] "Grin2c"        "Ppp1r3g"       "Etnppl"        "Apoe"         
 [29] "Timp4"         "Mfge8"         "Slc38a3"       "Phkg1"        
 [33] "Eva1a"         "Prdx6"         "Mt1"           "Plpp3"        
 [37] "Dbx2"          "Atp1a2"        "Chrdl1"        "Dio2"         
 [41] "Bcan"          "Hepacam"       "Itih3"         "Gldc"         
 [45] "Sox9"          "2900052N01Rik" "Nherf1"        "Prex2"        
 [49] "Oaf"           "Prodh"         "Rfx4"          "Myorg"        
 [53] "Mt2"           "Slc15a2"       "Glul"          "Cyp2j9"       
 [57] "Aldoc"         "Cmtm5"         "Fam107a"       "Gli3"         
 [61] "Pla2g7"        "Plcd4"         "Tril"          "Daam2"        
 [65] "Tst"           "Fam20a"        "Mertk"         "Slc27a1"      
 [69] "Lfng"          "Gstm1"         "Gm57100"       "Entpd2"       
 [73] "Sdc4"          "Cdh19"         "S100a16"       "Pdzph1"       
 [77] "Nat8f3"        "Gm57288"       "Ezr"           "Apln"         
 [81] "Vcam1"         "Cyp2d22"       "Kirrel2"       "Tmsb10"       
 [85] "Adhfe1"        "C030018K13Rik" "Clmn"          "Celrr"        
 [89] "Tlr3"          "Tsc22d4"       "Gm45774"       "Rftn2"        
 [93] "AW047730"      "Fzd2"          "Nkain4"        "Nat8f5"       
 [97] "Ica1"          "Pbxip1"        "B3galnt1"      "Fabp7"        
[101] "Mfsd2a"        "Kcnj10"        "Gramd2b"       "Cyp4f14"      
[105] "Slc4a4"        "Ednrb"         "Arhgef26"      "Nrarp"        
[109] "Gm13872"       "Gpam"          "Bmpr1b"        "Ptprz1"       
[113] "Klhdc7a"       "Appl2"         "Gpc5"          "Htra1"        
[117] "Slc6a6"        "Hprt1"         "Myo6"          "Gm38287"      
[121] "Igfbp2"        "Adora2b"       "Gpt"           "Gm973"        
[125] "Scara3"        "Abcg1"         "Gm3764"        "Srcin1"       
[129] "Ppp1r3c"       "Slc6a11"       "Sv2a"          "Nat8"         
[133] "Cd200"         "Zfp36l2"       "Fgfrl1"        "Cth"          
[137] "Atp9a"         "Gm266"         "Rmst"          "Hes5"         
[141] "Anks1b"        "Tppp"          "Aldh1l1"       "Scrg1"        
[145] "Arhgef19"      "A330048O09Rik" "Fscn1"         "AI849053"     
[149] "S100a1"        "Pdlim4"        "Slc1a2"        "Pax6"         
[153] "Selenop"       "Kcnj16"        "Phka1"         "Slc8a1"       
[157] "Lpgat1"        "Gng2"          "Rorb"          "Id4"          
[161] "Adap1"         "Gm48283"       "Epb41l3"       "Il18"         
[165] "Cideb"         "Ranbp3l"       "Pon2"          "Myt1l"        
[169] "Fxyd1"         "Dnajc6"        "Axl"           "Slc14a1"      
[173] "Crot"          "Nipal3"        "Ttpa"          "Map1b"        
[177] "Kcnq2"         "Dusp26"        "Atcay"         "Pdgfa"        
[181] "Nat8f7"        "Lgi4"          "C030005K06Rik" "Selenoh"      
[185] "Slc4a10"       "Slc7a2"        "Csrp1"         "Gm20421"      
[189] "Prkar2b"       "Slc6a17"       "Tmem100"       "Ndrg2"        
[193] "Reps2"         "Pdk4"          "Coro1a"        "Susd4"        
[197] "St8sia3os"     "Nalcn"         "Plekho2"       "Nmnat2"       
[201] "Folh1"         "Mtmr7"         "Tfcp2l1"       "B230209E15Rik"
[205] "Sez6l2"        "Dpp6"          "Ank3"          "St8sia3"      
[209] "Tmem151a"      "Jph3"          "Upp1"          "Scn2b"        
[213] "Kank1"         "Matk"          "Acss1"         "Basp1"        
[217] "Ly6h"          "Ica1l"         "Nipsnap1"      "Slc26a10"     
[221] "Sh3bp5"        "Sgk1"          "Gm13322"       "Gpr27"        
[225] "Myo10"         "Abracl"        "Csrnp3"        "Pmepa1"       
[22

In [88]:
if (unique) {
    all_genes <- unlist(pooled_ctype_genes)
    duplicates <- unique(names(table(all_genes)[table(all_genes) > 1]))

    # Remove markers that are not unique to a cell type

    pooled_ctype_genes <- lapply(pooled_ctype_genes, function(x) x[!(x %in% duplicates)])
}

In [89]:
data.frame(No.genes=lengths(pooled_ctype_genes)) %>% arrange(-`No.genes`)

,No.genes
,<int>
Peri,1568
Macrophage,395
Endo,304
Astro,282
VLMC,136
SMC,129
Oligo,105
Meis2,83
CR,61


#### Pairwise tests

In [90]:
pairwise_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream.RDS")

In [91]:
ctypes <- unique(sapply(strsplit(names(pairwise_res_list), "_vs_"), "[", 1))

pairwise_ctype_genes <- lapply(ctypes, function(target) {
    # Subset to pairwise tests with target cell type
    ctype_res_list <- pairwise_res_list[grep(paste0("^", target), names(pairwise_res_list))]

    # For each pairwise test, return genes that meet p-value threshold:
    pval_threshold <- .05 /length(ctype_res_list)
    ctype_genes_list <- lapply(ctype_res_list, function(df) {
        mask <- df$adj.P.Val < pval_threshold
        df[mask, 1]
    }) 

    # Restrict to genes that were identified in EVERY pairwise test
    Reduce(intersect, ctype_genes_list)
})
names(pairwise_ctype_genes) <- ctypes

In [92]:
data.frame(No.genes=lengths(pairwise_ctype_genes)) %>% arrange(-`No.genes`)

,No.genes
,<int>
Macrophage,333
Astro,288
Endo,237
Oligo,131
VLMC,85
SMC,60
Pvalb,31
Lamp5,22
CR,21


In [93]:
pairwise_ctype_genes <- pairwise_ctype_genes[lengths(pairwise_ctype_genes) > 0]

In [94]:
if (unique) {
    all_genes <- unlist(pairwise_ctype_genes)
    duplicates <- unique(names(table(all_genes)[table(all_genes) > 1]))

    # Remove markers that are not unique to a cell type

    pairwise_ctype_genes <- lapply(pairwise_ctype_genes, function(x) x[!(x %in% duplicates)])
}

pairwise_ctype_genes <- pairwise_ctype_genes[lengths(pairwise_ctype_genes) > 0]

In [95]:
pairwise_ctype_genes

$Astro
  [1] "Ttyh1"         "Kcnk1"         "Dtna"          "Ntsr2"        
  [5] "Gpr37l1"       "Cspg5"         "Cldn10"        "Bcan"         
  [9] "Slc1a3"        "Gja1"          "Plpp3"         "Mt1"          
 [13] "Scg3"          "Mt2"           "Ugp2"          "F3"           
 [17] "Phkg1"         "S1pr1"         "Gjb6"          "Fgfr3"        
 [21] "Nkain4"        "Acsl3"         "Slc1a2"        "Ptprz1"       
 [25] "Gm6145"        "Gm45774"       "Kcnn2"         "Aqp4"         
 [29] "Slc39a12"      "Lcat"          "Sox2"          "Acsbg1"       
 [33] "Hacd2"         "Cyp4f15"       "Nwd1"          "Prodh"        
 [37] "Mmd2"          "Glul"          "Ttyh3"         "Clu"          
 [41] "Slc6a11"       "Mlc1"          "Abhd3"         "Dio2"         
 [45] "Nat8f1"        "Msmo1"         "Dbx2"          "Pla2g7"       
 [49] "Cbs"           "Slc25a18"      "Hepacam"       "Acsl6"        
 [53] "Paqr7"         "Fam107a"       "Tril"          "Atp13a4"      
 [57] "Hes5"          "Grm3"          "Daam2"         "Itih3"        
 [61] "Btbd17"        "Aldoc"         "Frmd4a"        "Agpat5"       
 [65] "Grin2c"        "Fads2"         "Etnppl"        "Gldc"         
 [69] "Gpam"          "Ppp1r3g"       "AW047730"      "Paqr8"        
 [73] "Slitrk2"       "4933431E20Rik" "Gm45875"       "Eva1a"        
 [77] "Myorg"         "Chrdl1"        "Prex2"         "Lix1"         
 [81] "Gm7908"        "Plcd4"         "Sox9"          "Lfng"         
 [85] "Rfx4"          "Slc4a4"        "Sdc4"          "Gstm5"        
 [89] "Adhfe1"        "Prdx6"         "Slc7a10"       "Timp4"        
 [93] "Gnao1"         "Il18"          "Aldh1l1"       "Ednrb"        
 [97] "Tlcd1"         "9430041J12Rik" "Cdh19"         "Fam181b"      
[101] "Bmpr1b"        "Gpc5"          "Tst"           "Gpt"          
[105] "Gm15601"       "St3gal4"       "Fam228b"       "Ephx2"        
[109] "Fam20a"        "Tlr3"          "Gli3"          "AI849053"     
[113] "Mt3"           "2900052N01Rik" "Clmn"          "Soat1"        
[117] "Ppil6"         "Crot"          "Pax6"          "Ppp1r3c"      
[121] "Scara3"        "Acss1"         "Gm38287"       "Gm9934"       
[125] "Apln"          "Fbrsl1"        "Ndrg2"         "Sema4b"       
[129] "Mfap3l"        "Rftn2"         "Igdcc4"        "Gm57100"      
[133] "Lgi4"          "Sumf2"         "Slc15a2"       "Gm20421"      
[137] "Etv4"          "Abcb9"         "Ndp"           "Naaa"         
[141] "Rsrp1"         "Cntfr"         "Gm44257"       "Tom1l1"       
[145] "Pdzph1"        "Slc27a1"       "Cth"           "Serhl"        
[149] "Gabrg1"        "Gm43322"       "Gm3764"        "Gm57288"      
[153] "Fgfrl1"        "Gm13872"       "Chi3l1"        "Utp14b"       
[157] "Hopxos"        "Kirrel2"       "Phka1"         "Celrr"        
[161] "Cyp4f14"       "Pantr2"        "Nadk2"         "Washc2"       
[165] "Prr5"          "C030018K13Rik" "Pld2"          "Htra1"        
[169] "Cyp2j9"        "Cideb"         "Gm56959"       "Acot11"       
[173] "Nat8f5"        "Fgd6"          "Arhgef19"      "Kcnj16"       
[177] "Nim1k"         "Glud1"         "4930488L21Rik" "Nphp3"        
[181] "Gm43622"       "Cbr3"          "Ttpa"          "Nat8f3"       
[185] "Elovl2"        "Gna13"         "Hsd11b1"       "Gsap"         
[189] "9330159F19Rik" "Klhdc7a"       "1700003M07Rik" "Ntrk2"        
[193] "Smpd2"         "Gm40578"       "Gm26703"       "Kcnn3"        
[197] "Fxyd3"         "Vegfa"         "Rida"          "Entpd2"       
[201] "Mir99ahg"      "Gm266"         "Gm44895"       "1700071M16Rik"
[205] "Rasa2"         "Gm973"         "Nat8"          "Slc13a5"      
[209] "Appl2"         "Srebf1"        "C030037D09Rik" "C030005K06Rik"
[213] "Nat8f7"        "Gm43765"       "Id4"           "A330048O09Rik"
[217] "Sall3"         "Ddb2"          "Gm5089"        "Upp1"         
[221] "Gpld1"         "Caskin1"       "Ccdc190"       "Dhrs11"       
[225] "Fam228a"       "Prkd1"         "Cfap100"       "Sox21"        
[22

#### 1 vs. mean tests

In [96]:
mean_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream.RDS")

In [97]:
pval_threshold <- .05/length(mean_res_list)
lfc_threshold <- 6

mean_ctype_genes <- lapply(mean_res_list, function(df) {
    mask <- (df['adj.P.Val'] < pval_threshold) & (abs(df['logFC']) > lfc_threshold)
    df[mask, 1]
})
names(mean_ctype_genes) <- names(mean_res_list) 

In [98]:
if (unique) {
    all_genes <- unlist(mean_ctype_genes)
    duplicates <- unique(names(table(all_genes)[table(all_genes) > 1]))

    # Remove markers that are not unique to a cell type

    mean_ctype_genes <- lapply(mean_ctype_genes, function(x) x[!(x %in% duplicates)])
}

In [99]:
data.frame(No.genes=lengths(mean_ctype_genes)) %>% arrange(-`No.genes`)

,No.genes
,<int>
Peri,1279
Macrophage,355
Endo,325
Astro,213
Oligo,119
VLMC,110
SMC,94
CR,31
Meis2,30


### Enrichment results using 1 vs. pooled DE genes

In [100]:
pooled_enrichments_df <- get_module_enrichments(network_dir, pooled_ctype_genes, mod_def)

In [101]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

pooled_top_mods_df <- pooled_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [102]:
pooled_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Macrophage,3.192318e-267,1.690205e-262,pink,Bicor-None_signum0.803_minSize6_merge_ME_0.9_20151
Endo,9.571689e-232,1.013565e-227,greenyellow,Bicor-None_signum0.627_minSize5_merge_ME_0.9_20151
Astro,1.950997e-180,7.945962e-177,brown,Bicor-None_signum0.361_minSize10_merge_ME_0.9_20151
Oligo,2.215668e-125,3.554871e-122,red,Bicor-None_signum0.89_minSize3_merge_ME_0.9_20151
VLMC,2.746647e-124,4.277176e-121,lightgreen,Bicor-None_signum0.361_minSize8_merge_ME_0.9_20151
SMC,2.991812e-69,2.329478e-66,salmon,Bicor-None_signum0.627_minSize8_merge_ME_0.9_20151
Peri,1.055105e-35,5.032756e-33,brown,Bicor-None_signum0.89_minSize4_merge_ME_0.9_20151
NP,1.011897e-31,4.185618e-29,tan,Bicor-None_signum0.803_minSize3_merge_ME_0.9_20151
L6_CT,5.844798e-23,1.809700e-20,magenta4,Bicor-None_signum0.361_minSize5_merge_ME_0.9_20151


In [ ]:
if (unique) {
    write.csv(pooled_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(pooled_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}

### Enrichment results using pairwise DE genes

In [80]:
pairwise_enrichments_df <- get_module_enrichments(network_dir, pairwise_ctype_genes, mod_def)

ERROR: Error in fread(file.path(networks[i], kME_path), data.table = FALSE): input= must be a single character string containing a file name, a system command containing at least one space, a URL starting 'http[s]://', 'ftp[s]://' or 'file://', or, the input data itself containing at least one \n or \r


In [ ]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

pairwise_top_mods_df <- pairwise_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [ ]:
pairwise_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Astro,0.000000e+00,0.000000e+00,brown,Bicor-None_signum0.301_minSize5_merge_ME_0.9_20151
Endo,0.000000e+00,0.000000e+00,blue,Bicor-None_signum0.301_minSize10_merge_ME_0.9_20151
Macrophage,0.000000e+00,0.000000e+00,turquoise,Bicor-None_signum0.301_minSize10_merge_ME_0.9_20151
Oligo,1.487769e-187,1.638051e-184,green,Bicor-None_signum0.434_minSize10_merge_ME_0.9_20151
VLMC,3.682008e-140,3.093791e-137,black,Bicor-None_signum0.434_minSize4_merge_ME_0.9_20151
SMC,1.007654e-101,6.943971e-99,magenta,Bicor-None_signum0.301_minSize5_merge_ME_0.9_20151
Pvalb,5.772903e-45,3.233771e-42,coral4,Bicor-None_signum0.434_minSize8_merge_ME_0.9_20151
Sst,6.750495e-38,3.673957e-35,palevioletred3,Bicor-None_signum0.349_minSize6_merge_ME_0.9_20151
CR,1.932569e-35,9.952525e-33,darkorange,Bicor-None_signum0.434_minSize5_merge_ME_0.9_20151


In [ ]:
if (unique) {
    write.csv(pairwise_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(pairwise_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}

### Enrichment results using 1 vs. mean DE genes

In [ ]:
mean_enrichments_df <- get_module_enrichments(network_dir, mean_ctype_genes, mod_def)

In [ ]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

mean_top_mods_df <- mean_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [ ]:
mean_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Macrophage,0.000000e+00,0.000000e+00,turquoise,Bicor-None_signum0.301_minSize5_merge_ME_0.9_20151
Endo,1.728489e-247,9.110230e-244,blue,Bicor-None_signum0.301_minSize5_merge_ME_0.9_20151
Astro,6.085333e-234,2.539156e-230,blue,Bicor-None_signum0.434_minSize4_merge_ME_0.9_20151
Oligo,2.890408e-136,3.365712e-133,green,Bicor-None_signum0.301_minSize5_merge_ME_0.9_20151
VLMC,2.653974e-112,2.483872e-109,yellow,Bicor-None_signum0.719_minSize3_merge_ME_0.9_20151
Peri,1.342257e-89,9.811405e-87,purple,Bicor-None_signum0.719_minSize10_merge_ME_0.9_20151
SMC,4.055751e-64,2.840217e-61,magenta,Bicor-None_signum0.301_minSize5_merge_ME_0.9_20151
Sst,2.251725e-38,1.252735e-35,brown4,Bicor-None_signum0.434_minSize4_merge_ME_0.9_20151
NP,3.675325e-35,1.907018e-32,lightslateblue,Bicor-None_signum0.719_minSize4_merge_ME_0.9_20151


In [ ]:
if (unique) {
    write.csv(mean_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(mean_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}

### Compare enrichment results

In [ ]:
pooled_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", pooled_top_mods_df$Network))
pairwise_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", pairwise_top_mods_df$Network))
mean_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", mean_top_mods_df$Network))

cols <- c("Cell_type", "Qval", "Module", "Network_short")

all_top_mods_df <- merge(
    merge(
        pooled_top_mods_df[,cols], 
        pairwise_top_mods_df[,cols], 
        by="Cell_type", all=TRUE
    ), 
    mean_top_mods_df[,cols], by="Cell_type", all=TRUE
)

In [ ]:
all_top_mods_df <- all_top_mods_df[,order(colnames(all_top_mods_df))]
all_top_mods_df

Cell_type,Module,Module.x,Module.y,Network_short,Network_short.x,Network_short.y,Qval,Qval.x,Qval.y
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
Astro,blue,blue,brown,signum0.434_minSize4,signum0.434_minSize4,signum0.301_minSize5,2.539156e-230,5.401352e-186,0.000000e+00
CR,darkorange,darkorange,darkorange,signum0.434_minSize4,signum0.434_minSize4,signum0.434_minSize5,1.484977e-28,1.548719e-22,9.952525e-33
Endo,blue,blue,blue,signum0.301_minSize5,signum0.301_minSize5,signum0.301_minSize10,9.110230e-244,2.311671e-256,0.000000e+00
L2_3_IT,thistle2,mediumpurple,thistle1,signum0.719_minSize4,signum0.434_minSize8,signum0.719_minSize3,6.240666e-13,1.018673e-08,9.610927e-13
L4,mistyrose4,NA,NA,signum0.349_minSize5,NA,NA,2.130419e-02,NA,NA
L5_IT,gray7,steelblue,steelblue,signum0.301_minSize5,signum0.349_minSize6,signum0.349_minSize6,4.572261e-08,7.104840e-12,3.872045e-18
L5_PT,skyblue3,ivory,palevioletred3,signum0.719_minSize6,signum0.301_minSize5,signum0.434_minSize4,6.632453e-11,1.052353e-08,1.105092e-05
L6_CT,coral4,salmon2,mediumpurple1,signum0.349_minSize6,signum0.434_minSize10,signum0.719_minSize3,4.954733e-15,1.325915e-16,9.055589e-14
L6_IT,khaki4,mediumpurple4,khaki4,signum0.719_minSize3,signum0.301_minSize10,signum0.719_minSize3,1.409707e-20,2.118107e-06,1.947953e-20


### Select the module from the lowest Qval per cell type

In [ ]:
cols <- c("Cell_type", "Qval", "Module", "Network_short")

In [ ]:
# Select the module from the lowest Qval per cell type

pooled_top_mods_df$DE_Test <- "Pooled"
pairwise_top_mods_df$DE_Test <- "Pairwise"
mean_top_mods_df$DE_Test <- "Mean"

all_top_mods_df <- rbind(pooled_top_mods_df, pairwise_top_mods_df, mean_top_mods_df)

In [ ]:
top_qval_mods_df <- all_top_mods_df %>%
    group_by(Cell_type) %>%
    slice_min(Qval)

top_qval_mods_df[, c(cols, "DE_Test")] %>% arrange(Qval)

Cell_type,Qval,Module,Network_short,DE_Test
<chr>,<dbl>,<chr>,<chr>,<chr>
Astro,0.000000e+00,brown,signum0.301_minSize5,Pairwise
CR,9.952525e-33,darkorange,signum0.434_minSize5,Pairwise
Endo,0.000000e+00,blue,signum0.301_minSize10,Pairwise
L2_3_IT,6.240666e-13,thistle2,signum0.719_minSize4,Mean
L4,2.130419e-02,mistyrose4,signum0.349_minSize5,Mean
L5_IT,3.872045e-18,steelblue,signum0.349_minSize6,Pairwise
L5_PT,6.632453e-11,skyblue3,signum0.719_minSize6,Mean
L6_CT,1.325915e-16,salmon2,signum0.434_minSize10,Pooled
L6_IT,1.409707e-20,khaki4,signum0.719_minSize3,Mean


In [ ]:
top_qval_mods_df

Network,kME_path,ME_path,Pval,Cell_type,Module,Qval,Network_short,DE_Test
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>
Bicor-None_signum0.301_minSize5_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.301_minSize5_merge_ME_0.9_20151/kME_table_08-19-57.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.301_minSize5_merge_ME_0.9_20151/Module_eigengenes_08-19-57.csv,0.000000e+00,Astro,brown,0.000000e+00,signum0.301_minSize5,Pairwise
Bicor-None_signum0.434_minSize5_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.434_minSize5_merge_ME_0.9_20151/kME_table_06-26-24.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.434_minSize5_merge_ME_0.9_20151/Module_eigengenes_06-26-24.csv,1.932569e-35,CR,darkorange,9.952525e-33,signum0.434_minSize5,Pairwise
Bicor-None_signum0.301_minSize10_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.301_minSize10_merge_ME_0.9_20151/kME_table_09-38-44.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.301_minSize10_merge_ME_0.9_20151/Module_eigengenes_09-38-44.csv,0.000000e+00,Endo,blue,0.000000e+00,signum0.301_minSize10,Pairwise
Bicor-None_signum0.719_minSize4_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.719_minSize4_merge_ME_0.9_20151/kME_table_05-50-06.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.719_minSize4_merge_ME_0.9_20151/Module_eigengenes_05-50-06.csv,2.037804e-15,L2_3_IT,thistle2,6.240666e-13,signum0.719_minSize4,Mean
Bicor-None_signum0.349_minSize5_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.349_minSize5_merge_ME_0.9_20151/kME_table_07-10-33.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.349_minSize5_merge_ME_0.9_20151/Module_eigengenes_07-10-33.csv,2.140162e-04,L4,mistyrose4,2.130419e-02,signum0.349_minSize5,Mean
Bicor-None_signum0.349_minSize6_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.349_minSize6_merge_ME_0.9_20151/kME_table_07-35-20.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.349_minSize6_merge_ME_0.9_20151/Module_eigengenes_07-35-20.csv,1.055042e-20,L5_IT,steelblue,3.872045e-18,signum0.349_minSize6,Pairwise
Bicor-None_signum0.719_minSize6_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_20SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.719_minSize6_merge_ME_0.9_20151/kME_table_05-57-43.csv,/mnt/lareaulab/reliscu/

In [182]:
if (unique) {
    write.csv(top_qval_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(top_qval_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}
